<a href="https://colab.research.google.com/github/joyboy129/federated/blob/main/Federated_learning_entropia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [74]:
#@test {"skip": true}

!pip install --quiet --upgrade tensorflow-federated
import nest_asyncio
nest_asyncio.apply()

In [75]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import MinMaxScaler


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/first_file.csv", sep=',', encoding='latin-1')
df.head()

In [ ]:
# Run for Bigdata only
# df.rename(columns={"ElectronÃ©gativitÃ©": "Electronégativité"}, inplace=True)
# df.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
df.dtypes
# df=df.iloc[:50000]
# Run for small data
df.rename(columns={"ElectronÃ©gativitÃ©": "Electronégativité"}, inplace=True)

df.columns
df.drop(['Alloy', 'Longue Phase', 'Ag', 'Al', 'Cd', 'Ga', 'Mg', 'Pt', 'Sb',
       'Sn', 'Ti', 'Zn', 'Zr', 'B', 'Ca', 'Ce', 'Co', 'Cr', 'Cu', 'Er', 'Fe',
       'In', 'La', 'Li', 'Mn', 'Mo', 'Nd', 'Ni', 'Re', 'Ru', 'Sm', 'V', 'Yb',
       'Au', 'Be', 'Bi', 'Pb', 'Nb', 'Si', 'Dy', 'Ge', 'Hf', 'W', 'Pd', 'Sr',
       'Na', 'Y', 'Ta', 'Gd', 'Rh', 'P', 'C', 'Ho', 'Pr', 'N', 'Tb', 'Sc',
       'Tm', 'Lu', 'Ir', 'Os', 'As', 'Tc'], axis=1, inplace=True)


In [ ]:
# client1 = 25
# client2 = 25
# client3 = 25
# client4 = 25
df["Etat d'entropie"] = df["Etat d'entropie"].map({'H': 1, 'L': 0})
df["Etat d'entropie"].value_counts()

In [ ]:
column_names=list(df.columns)
print(column_names)

In [ ]:
# import random
# client1 = int((client1/100) *df.shape[0])
# client2 = int((client2/100) *df.shape[0])
# client3 = int((client3/100) *df.shape[0])
# client4 = df.shape[0] - client1 - client2 - client3
# my_list = [1] * client1 + [2] * client2 + [3] * client3 + [4] * client4
# random.shuffle(my_list)
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

client_distributions = {
    1: {'percentage': 0.25, 'distribution': [0.60, 0.40]},
    2: {'percentage': 0.25, 'distribution': [0.50, 0.50]},
    3: {'percentage': 0.25, 'distribution': [0.70, 0.30]},
    4: {'percentage': 0.25, 'distribution': [0.80, 0.20]},
}
def distribute_dataframe(df, client_distributions):
    np.random.seed(42)

    # Separate the dataframe into 0s and 1s
    df0 = df[df['Etat d\'entropie'] == 0]
    df1 = df[df['Etat d\'entropie'] == 1]

    # Perform undersampling
    # if len(df0) > len(df1):
    #     df0 = df0.sample(len(df1))
    # else:
    #     df1 = df1.sample(len(df0))

    # Combine back to a single dataframe

    distributed_df = pd.DataFrame()

    for client_id, client_info in client_distributions.items():
        client_percentage = client_info['percentage']
        class_distribution = client_info['distribution']

        # Calculate the exact number of rows for this client
        n_rows = int(client_percentage * len(df))

        # Calculate desired number of samples for each class
        n0 = int(class_distribution[0] * n_rows)
        n1 = n_rows - n0

        # Check if there are enough samples for each class
        if n0 > len(df0):
            n0 = len(df0)
        if n1 > len(df1):
            n1 = len(df1)

        # Sample the rows for this client
        tmp_df0 = df0.sample(n=n0, random_state=42)
        tmp_df1 = df1.sample(n=n1, random_state=42)

        # Drop these rows from the original dataframes
        df0 = df0.drop(tmp_df0.index)
        df1 = df1.drop(tmp_df1.index)

        # Add the client id and combine the samples
        tmp_df = pd.concat([tmp_df0, tmp_df1])
        tmp_df['id'] = client_id

        distributed_df = pd.concat([distributed_df, tmp_df])


    return distributed_df

df =distribute_dataframe(df, client_distributions)
# Create a copy of the data excluding the specified column

# Prepare the 'id' column
# ids = np.random.choice([1,2,3,4], size=len(df), p=list(id_percentages.values()))
# df['id'] = ids

# # Prepare the "Etat d'entropie" column
# for id_val, percentages in entropy_percentages.items():
#     size = len(df[df['id'] == id_val])
#     entropy_values = np.random.choice([0,1], size=size, p=list(percentages.values()))
#     df.loc[df['id'] == id_val, 'Etat d\'entropie'] = entropy_values
df.head()

In [ ]:

def normalize_columns(df, columns_to_normalize):
    # Create a copy of the original dataframe
    normalized_df = df.copy()

    # Select the columns to normalize
    data_to_normalize = normalized_df[columns_to_normalize]

    # Create an instance of the scaler
    scaler = StandardScaler()

    # Fit the scaler to the selected columns
    scaler.fit(data_to_normalize)

    # Normalize the selected columns
    normalized_data = scaler.transform(data_to_normalize)

    # Update the normalized values in the dataframe
    normalized_df[columns_to_normalize] = normalized_data

    return normalized_df
df=normalize_columns(df,['A', 'delta', 'TM', 'DELTA_TM', 'Electronégativité', 'Delta_Electro-', 'SID', 'VEC', 'Delta VEC', 'K', 'Delta K'])
df.head()

In [ ]:
df.columns

In [ ]:
from scipy.stats import gaussian_kde
import seaborn as sns
import matplotlib.pyplot as plt


def pair_plot(df):
  etat_entropie_values = [0, 1]

# Plotting the density plots
  id_values = [1, 2, 3,4]

# Plotting the density plots
  num_rows = 11
  num_cols = 4
  num_plots = num_rows * num_cols

  fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 36))
  axes = axes.flatten()

  for i, col in enumerate(df.columns):
    if col != 'id' and col != 'Etat d\'entropie':
        for j, id_value in enumerate(id_values):
            ax = axes[(i * len(id_values)) + j]
            for etat_entropie in [0, 1]:
                filtered_data = df[df['Etat d\'entropie'] == etat_entropie]
                if not filtered_data.empty:
                    filtered_data[col].plot(kind='density', label=f'Etat Entropie = {etat_entropie}', ax=ax)
            filtered_data = df[(df['Etat d\'entropie'] == 0) | (df['Etat d\'entropie'] == 1)]
            if not filtered_data.empty:
                filtered_data[col].plot(kind='density', label='All Data', ax=ax, linestyle='--')
            ax.set_title(f'Density Plot - {col}, id = {id_value}')
            ax.legend()

  plt.tight_layout()
  plt.show()
pair_plot(df)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a pairplot
for i in [1,2,3,4]:
  # pair_plot = sns.pairplot(df[df['id'] == i], hue='Etat d\'entropie')
  # pair_plot.savefig("/content/drive/MyDrive/Figures/Pairplots/pairplot"+str(i)+".png")



# Save the plot as a PNG file
# pair_plot.savefig("pairplot.png")


In [ ]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

In [ ]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
np.random.seed(0)
tff.federated_computation(lambda: 'Hello, World!')()

In [ ]:
df.dtypes

In [ ]:
client_id_colname = 'id' # the column that represents client ID
SHUFFLE_BUFFER = 100
NUM_EPOCHS = 10

In [ ]:
# split client id into train and test clients
train_frac = 0.9
client_ids = df[client_id_colname].unique()
train_client_ids = [1,2,3]
test_client_ids = [4]
print(client_ids)

In [ ]:
train_client_ids

In [ ]:
def create_tf_dataset_for_client_fn(client_id):
  # a function which takes a client_id and returns a
  # tf.data.Dataset for that client
    client_data = df[df[client_id_colname] == client_id]
    client_data = client_data.drop(columns=["id"])
    dataset = tf.data.Dataset.from_tensor_slices(client_data.to_dict('list'))
    dataset = dataset.shuffle(SHUFFLE_BUFFER).batch(1).repeat(NUM_EPOCHS)
    return dataset

In [ ]:
train_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=train_client_ids,
        serializable_dataset_fn=create_tf_dataset_for_client_fn
    )

#
test_data = tff.simulation.datasets.ClientData.from_clients_and_tf_fn(
        client_ids=test_client_ids,
        serializable_dataset_fn=create_tf_dataset_for_client_fn
    )

In [ ]:
train_data.element_type_structure

In [ ]:
example_dataset = train_data.create_tf_dataset_for_client(
    train_data.client_ids[0])

example_element = next(iter(example_dataset))


In [ ]:
NUM_CLIENTS = 4
NUM_EPOCHS = 100
BATCH_SIZE = 32
SHUFFLE_BUFFER = 1000
PREFETCH_BUFFER = 10

def preprocess(dataset):

        def batch_format_fn(element):
            output = tf.concat([
                tf.expand_dims(element['Electronégativité'],1)
                ,tf.expand_dims(element['Delta_Electro-'],1)
                ,tf.expand_dims(element['SID'],1)
                ,tf.expand_dims(element['VEC'],1)
                ,tf.expand_dims(element['Delta VEC'],1)
                ,tf.expand_dims(element[ 'K'],1)
                ,tf.expand_dims(element['Delta K'],1)
                ,tf.expand_dims(element['A'],1)
                ,tf.expand_dims(element['delta'],1)
                ,tf.expand_dims(element['TM'],1)
                ,tf.expand_dims(element['DELTA_TM'],1)], axis=1)
            return collections.OrderedDict(x= tf.reshape(output, [1, 11] )
                                           ,
                                           y= tf.reshape(element["Etat d'entropie"], [1,1]) )

        return dataset.map(batch_format_fn)

In [ ]:
preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch

In [ ]:
def make_federated_data(client_data, client_ids):
    return [
      preprocess(client_data.create_tf_dataset_for_client(x))
      for x in client_ids
  ]

In [ ]:
sample_clients = train_data.client_ids[0:NUM_CLIENTS]
sample_clients_test=test_data.client_ids[0:NUM_CLIENTS]
federated_train_data = make_federated_data(train_data, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

In [ ]:

def create_keras_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Reshape(target_shape=(11, 1), input_shape=(11,)),
    # Reshape input to (11, 1)
    tf.keras.layers.Conv1D(20, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=8),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(15, kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(5, kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(1, kernel_initializer='lecun_normal', activation='relu')
    ])
# RNN
# def create_keras_model():
#     return tf.keras.models.Sequential([
#         tf.keras.layers.Reshape(target_shape=(11, 1), input_shape=(11,)),
#         tf.keras.layers.SimpleRNN(32),
#         tf.keras.layers.Dense(85, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(60, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(25, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(10, kernel_initializer='zeros'),
#         tf.keras.layers.Softmax(),
#     ])
# def create_keras_model():
#     return tf.keras.models.Sequential([
#         tf.keras.layers.Reshape(target_shape=(11, 1), input_shape=(11,)),
#         tf.keras.layers.LSTM(32),
#         tf.keras.layers.Dense(85, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(60, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(25, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(10, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(32, kernel_initializer='lecun_normal',activation='selu'),
#     ])
# def create_keras_model():
#     return tf.keras.models.Sequential([
#         tf.keras.layers.Reshape(target_shape=(11, 1), input_shape=(11,)),
#         tf.keras.layers.GRU(32),
#         tf.keras.layers.Dense(30, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(25, kernel_initializer='zeros'),
#         tf.keras.layers.Dense(1, kernel_initializer='zeros'),
#         tf.keras.layers.Softmax(),
#     ])

model=create_keras_model()
tf.keras.utils.plot_model(
model,
to_file="model_ann.png",
show_shapes=True,
show_dtype=False,
show_layer_names=True,
rankdir="TB",
expand_nested=True,
dpi=96,
layer_range=None,
show_layer_activations=True,
)

In [ ]:
def model_fn():
  # We _must_ create a new model here, and _not_ capture it from an external
  # scope. TFF will call this within different graph contexts.
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
    keras_model=keras_model,
    input_spec=preprocessed_example_dataset.element_spec,
    loss=tf.keras.losses.Poisson(reduction="auto", name="poisson"),
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(),
        tf.keras.metrics.TruePositives(),
        tf.keras.metrics.TrueNegatives(),
        tf.keras.metrics.FalsePositives(),
        tf.keras.metrics.FalseNegatives()
    ]
)

In [ ]:
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam())



In [ ]:
str(iterative_process.initialize.type_signature)

In [ ]:
state = iterative_process.initialize()

In [ ]:
df.dtypes

In [ ]:
state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics))

In [ ]:
NUM_ROUNDS = 4

acc = []
for round_num in range(2, NUM_ROUNDS):
    state, metrics = iterative_process.next(state, federated_train_data)
    print('round {:2d}, metrics={}'.format(round_num, metrics))
    acc.append(float( metrics['client_work']['train']['sparse_categorical_accuracy']))

In [ ]:

import matplotlib.pyplot as plt
import scipy.stats as stats

# Assuming you have an array named acc

# Plotting the array
plt.plot(acc)
plt.xlabel('Index')
plt.ylabel('Accuracy')
plt.title('Accuracy Plot')
plt.show()

# Creating a boxplot of the array
plt.boxplot(acc)
plt.ylabel('Accuracy')
plt.title('Accuracy Boxplot')
plt.show()



In [ ]:
max(acc)

In [ ]:
#@test {"skip": true}
logdir = "/tmp/logs/scalars/training/"
try:
  tf.io.gfile.rmtree(logdir)  # delete any previous results
except tf.errors.NotFoundError as e:
  pass # Ignore if the directory didn't previously exist.
summary_writer = tf.summary.create_file_writer(logdir)
train_state = iterative_process.initialize()

In [ ]:
rm -rf ./logs/


In [ ]:
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


Evaluation process

In [ ]:

# Create a model function

# Use this model function to create a training process
evaluation_process = tff.learning.algorithms.build_fed_eval(model_fn)

In [ ]:
print(evaluation_process.next.type_signature.formatted_representation())

In [ ]:
evaluation_state = evaluation_process.initialize()
model_weights = iterative_process.get_model_weights(state)
evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_train_data)

In [ ]:
str(evaluation_output.metrics)

In [ ]:
evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['true_positives']


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
tp = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['true_positives'])
tn = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['true_negatives'])
fp = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['false_positives'])
fn = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['false_negatives'])

cm = np.array([[tp, fp],
               [fn, tn]])

# labels
labels = ['Positive', 'Negative']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
federated_test_data = make_federated_data(test_data, test_data.client_ids)

len(federated_test_data), federated_test_data[0]

In [ ]:
evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

In [ ]:
tp = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['true_positives'])
tn = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['true_negatives'])
fp = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['false_positives'])
fn = int(evaluation_output.metrics["client_work"]["eval"]['total_rounds_metrics']['false_negatives'])

cm = np.array([[tp, fp],
               [fn, tn]])

# labels
labels = ['Positive', 'Negative']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

Deep learning


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
X = data.drop(columns=['Etat d\'entropie', 'id'])
y = data['Etat d\'entropie']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and compile the model
model = create_keras_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10)

# Test the model
y_pred = (model.predict(X_test) > 0.9).astype("int32")

# Plot confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()